In [3]:
import os
import numpy as np
from PIL import Image

# 1. إعداد المسارات (تأكد من وجود مجلدات للأشخاص داخل مسار Omar)
BASE_PATH = '/storage/emulated/0/AI_Projects/NLP_Course/Dataset'

def load_and_preprocess(path, size=(64, 64)):
    faces = []
    labels = []
    user_names = []
    
    print("جاري معالجة الصور...")
    for idx, person in enumerate(os.listdir(path)):
        person_path = os.path.join(path, person)
        if not os.path.isdir(person_path): continue
        
        user_names.append(person)
        for img_name in os.listdir(person_path):
            try:
                img = Image.open(os.path.join(person_path, img_name)).convert('L')
                img = img.resize(size)
                faces.append(np.array(img).flatten())
                labels.append(idx)
            except:
                continue
    return np.array(faces), np.array(labels), user_names

# 2. تدريب النموذج (Eigenfaces البدائي المحسن)
X, y, names = load_and_preprocess(BASE_PATH)

if len(X) == 0:
    print("خطأ: لم يتم العثور على صور!")
else:
    # حساب "متوسط الوجوه" لكل شخص
    print("جاري إنشاء البصمات الحيوية...")
    unique_labels = np.unique(y)
    face_signatures = []
    
    for label in unique_labels:
        # استخراج صور شخص واحد وحساب المتوسط لها لتكوين "بصمة وجه"
        person_faces = X[y == label]
        signature = np.mean(person_faces, axis=0)
        face_signatures.append(signature)
    
    face_signatures = np.array(face_signatures)
    print(f"تم التدريب! بصمات الوجوه جاهزة لـ {len(names)} أشخاص.")

# 3. دالة التوقع (Prediction)
def predict_face(image_path, signatures, names, size=(64, 64)):
    img = Image.open(image_path).convert('L').resize(size)
    img_flat = np.array(img).flatten()
    
    # حساب المسافة بين الصورة المدخلة وكل البصمات المخزنة
    distances = np.linalg.norm(signatures - img_flat, axis=1)
    best_match = np.argmin(distances)
    
    return names[best_match], distances[best_match]

print("جاهز للاختبار!")


جاري معالجة الصور...
جاري إنشاء البصمات الحيوية...
تم التدريب! بصمات الوجوه جاهزة لـ 6 أشخاص.
جاهز للاختبار!


In [7]:
# 1. حدد مسار الصورة التي تريد اختبارها (تأكد من وجود صورة في هذا المسار)
# يمكنك تغيير 'test.jpg' لاسم أي صورة عندك في مجلد Omar
TEST_IMAGE_PATH = '/storage/emulated/0/AI_Projects/Dataset/Muhammad/face1.jpg'

try:
    # 2. استدعاء دالة التوقع
    name, confidence = predict_face(TEST_IMAGE_PATH, face_signatures, names)

    # 3. عرض النتيجة
    print(f"--- نتيجة الفحص ---")
    print(f"الشخص المتوقع: {name}")
    print(f"مستوى الاختلاف (Distance): {confidence:.2f}")
    print("------------------")
    
    # ملاحظة: كلما قل رقم Distance، كانت الثقة في النتيجة أكبر
    if confidence < 3000: # يمكنك تعديل هذا الرقم بناءً على تجاربك
        print("النتيجة: تم التعرف على الوجه بنجاح!")
    else:
        print("النتيجة: غير متأكد تماماً، ربما الشخص غير مسجل أو الإضاءة مختلفة.")

except Exception as e:
    print(f"حدث خطأ أثناء الاختبار: {e}")
    print("تأكد من أن مسار الصورة الصحيح وأن الملف موجود فعلاً.")


--- نتيجة الفحص ---
الشخص المتوقع: Omar
مستوى الاختلاف (Distance): 4907.89
------------------
النتيجة: غير متأكد تماماً، ربما الشخص غير مسجل أو الإضاءة مختلفة.
